# Sumatra export


This notebook file was written by Andrey Moskaelnko from Purdue University (amoskale@purdue.edu)

In [ ]:
#This cell will increase the width of the cells to fit more of the screen
#adjust the precentage value as you wish
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import numpy as np
import sympy
import fipy as fp
from fipy import numerix as nmx
import matplotlib.pyplot as plt
import os
import json
import sys

In [ ]:
os.getcwd()

In [ ]:

#record the volume integral of the free energy 
# equivalent to the average value of the free energy for any cell,
# multiplied by the number of cells and the area of each cell
# (since this is a 2D domain)        
# bulk free energy density
def f_0(c):
    return rho_s*((c - c_alpha)**2)*((c_beta-c)**2)
def f_0_var(c_var):
    return 2*rho_s*((c_alpha - c_var)**2 + 4*(c_alpha - c_var)*(c_beta - c_var) + (c_beta - c_var)**2)
# free energy
def f(c):
    return (f_0(c)+ .5*kappa*(c.grad.mag)**2)

The next block will create a python file in your directory to run the 1a, 1b, 1c, problems from https://pages.nist.gov/chimad-phase-field/hackathon1/

In [ ]:
%%writefile fipy_hackathon_1.py
        
#read in the parameter file
jsonfile = sys.argv[1] 
if jsonfile:
    with open(jsonfile, 'rb') as ff:
        params = json.load(ff)
        
else:
    print "***No parameter file found! ***"
    
print 'my params:', params

#extract the parameters
N = params.get('N', 20)  
total_steps = params.get('steps', 2)
sumatra_label = params.get('sumatra_label', '')
total_sweeps = params.get ('sweeps', 2)
duration = params.get('duration', 500)
problem = params.get('problem', 'a')
tolerance = params.get('tolerance',0.1)
solverParam = params.get('solver', 'PCG')


c, rho_s, c_alpha, c_beta = sympy.symbols("c_var rho_s c_alpha c_beta")
f_0 = rho_s * (c - c_alpha)**2 * (c_beta - c)**2

sympy.diff(f_0, c, 2)

# command format:


if (problem == "a"):
    print "Creating mesh for problem a"
    mesh = fp.PeriodicGrid2D(nx=N, ny=N, dx=200.0/float(N), dy=200.0/float(N))
    
elif (problem == "b"):
    print "Creating mesh for problem b"
    mesh = fp.Grid2D(nx=N, ny=N, dx=200.0/float(N), dy=200.0/float(N))
    
elif (problem == "c"):
    print "Creating mesh for problem c"
    Lx=20.
    Ly=100.0
    dxT=100./N
    dyT=100./N
    nxT=N
    nyT=N/5
    mesh = fp.Grid2D(nx=N / 5, ny=N, dx=20./(N/5), dy=100./N) + (fp.Grid2D(nx=N, ny=N / 5, dx=100./N, dy=20./(N/5)) + [[-40],[100]])
#    mesh = fp.Grid2D(dx=0.5, dy=0.5, nx=40, ny=200) + (fp.Grid2D(dx=0.5, dy=0.5, nx=200, ny=40) + [[-40],[100]])
    mesh = fp.Grid2D(Lx=20., Ly=100.0, nx=N / 5, ny=N) + (fp.Grid2D(Ly=20.0, Lx=100.0, nx=N, ny=N / 5) + [[-40],[100]])
       
elif (problem == "d"):
    print "Creating mesh for problem d"
    r = float(sys.argv[2])
    numCellsDesired = int(sys.argv[3])
    epsilon = 0.05
    cellSize = 16 * np.pi * r**2 / (nmx.sqrt(3.) * numCellsDesired)
    cellSize = nmx.sqrt(cellSize)
    
    substring1 = '''
    radius = {0};
    cellSize = {1};
    '''.format(r, round(cellSize, 6))

    mesh = fp.Gmsh2DIn3DSpace(substring1 + '''

    // create inner 1/8 shell
    Point(1) = {0, 0, 0, cellSize};
    Point(2) = {-radius, 0, 0, cellSize};
    Point(3) = {0, radius, 0, cellSize};
    Point(4) = {0, 0, radius, cellSize};
    Circle(1) = {2, 1, 3};
    Circle(2) = {4, 1, 2};
    Circle(3) = {4, 1, 3};
    Line Loop(1) = {1, -3, 2};
    Ruled Surface(1) = {1};

    // create remaining 7/8 inner shells
    t1[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{1};}};
    t2[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{1};}};
    t3[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{1};}};
    t4[] = Rotate {{0,1,0},{0,0,0},-Pi/2} {Duplicata{Surface{1};}};
    t5[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{t4[0]};}};
    t6[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{t4[0]};}};
    t7[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{t4[0]};}};

    // create entire inner and outer shell
    Surface Loop(100)={1, t1[0],t2[0],t3[0],t7[0],t4[0],t5[0],t6[0]};
    ''', order=2.0).extrude(extrudeFunc=lambda r: 1.01*r)

c_alpha = 0.3
c_beta = 0.7
kappa = 2.0
M = 5.0
c_0 = 0.5
epsilon = 0.01
rho_s = 5.0



# solution variable
c_var = fp.CellVariable(mesh=mesh, name=r"$c$", hasOld=True)

# array of sample c-values: used in f versus c plot
vals = np.linspace(-.1, 1.1, 1000)

if (problem == 'a' or 'b' or 'c'):
    # 2D mesh coordinates
    x, y = np.array(mesh.x), np.array(mesh.y)
    # initial value for square and T domains
    c_var[:] = c_0 + epsilon * (np.cos(0.105 * x) * np.cos(0.11 * y) + (np.cos(0.13 * x) * np.cos(0.087 * y))**2 + np.cos(0.025 * x - 0.15 * y) * np.cos(0.07 * x - 0.02 * y))
if (problem == 'd'):
    print "number of cells: " , mesh.numberOfCells
    # 3D mesh coordinates
    x, y, z = np.array(mesh.x), np.array(mesh.y), np.array(mesh.z)
    
    # convert from rectangular to spherical coordinates
    theta = fp.CellVariable(name=r"$\theta$", mesh=mesh)
    theta = nmx.arctan2(z, nmx.sqrt(x**2 + y**2))
    phi = fp.CellVariable(name=r"$\phi$", mesh=mesh)
    phi = nmx.arctan2(y, x)
     
    # initial value for spherical domain
    c_var[:]  = c_0 + epsilon * ((np.cos(8*theta))*(np.cos(15*phi)) + ((np.cos(12*theta))*(np.cos(10*phi)))**2 + ((np.cos(2.5*theta - 1.5*phi))*(np.cos(7*theta - 2*phi))))
    

#Method for making sure we save .mpz.npz at specified dump_times
def calc_dt(elapsed_time, dt, dt_old, dump_to_file, dump_times, filename):
    if dump_to_file: #if this is true, we have alreay saved the necessary .mpz.npz file
        dt = dt_old #reset back to normal dt
        dt = dt * 1.1 #continue as normal
        dump_to_file = False
        # filename = '1a_{0}_step{1}_data_time-{2:.2f}.npz'.format(N, str(steps).rjust(6, '0'), elapsed+dt)
    else:
        dt_old = dt
        dt = dt * 1.1
        # import ipdb; ipdb.set_trace()

        if len(dump_times) > 0:
            if (elapsed_time + dt) >= dump_times[0]:
                dt = dump_times[0] - elapsed_time
                dump_to_file = True
                
                #dump_time files will have .mpz.npz extension
                # filename = '1a_{0}_step{1}_data_time-{2:.2f}.mpz.npz'.format(N, str(steps).rjust(6, '0'), elapsed+dt)
                del dump_times[0]

    return dt, dt_old, dump_times, dump_to_file, filename
    
# save elapsed time and free energy at each data point
time_data = []
cvar_data = []
f_data = []
# checks whether a folder for the pickles from this simulation exists
# if not, creates one in the home directory
filepath = os.path.join(os.getcwd(), sumatra_label)


# solver equation    
eqn = fp.TransientTerm(coeff=1.) == fp.DiffusionTerm(M * f_0_var(c_var)) - fp.DiffusionTerm((M, kappa))

elapsed = 0.0
steps = 0
dt = 0.01
dump_times = [1.0, 5.0, 10.0, 20.0, 100.0, 200, 500, 1000, 2000, 3000, 10000] #the specific elapsed times when we want to save out .mpz.npz files
dt_old = dt
dump_to_file = False
filename = 'Andrey Moskalenko'


c_var.updateOld()
if (solverParam = "PCG"):
    from fipy.solvers.pysparse import LinearPCGSolver as Solver
if (solverParam = "LU"):
    from fipy.solvers.pysparse import LinearLUSolver as Solver
solver = Solver()
print "Starting Solver."
while (steps <= total_steps) and elapsed <= duration:
    res0 = eqn.sweep(c_var, dt=dt, solver=solver)

    
    for sweeps in range(total_sweeps):
        res = eqn.sweep(c_var, dt=dt, solver=solver)
                    
    if res < res0 * tolerance:  
        # anything in this loop will only be executed every 10 steps
        if dump_to_file or (steps%10==0):
            print steps
            print elapsed 
            print "Saving data"
            if dump_to_file: filename = '1{3}_{0}_step{1}_data_time-{2:.2f}.mpz.npz'.format(N, str(steps).rjust(6, '0'), elapsed, problem)
            else: filename = '1{3}_{0}_step{1}_data_time-{2:.2f}.npz'.format(N, str(steps).rjust(6, '0'), elapsed, problem)
            if (problem == 'a') or (problem == 'b'):
                np.savez(os.path.join(filepath,filename),
                     c_var_array=np.array(c_var),
                     dt=dt,
                     elapsed=elapsed,
                     steps=steps,
                     dx=c_var.mesh.dx,
                     dy=c_var.mesh.dy,
                     nx=c_var.mesh.nx,
                     ny=c_var.mesh.ny,
                     sweeps = total_sweeps,
                     tolerance = tolerance)
            if (problem == 'c'):
                np.savez(os.path.join(filepath,filename),
                     c_var_array=np.array(c_var),
                     dt=dt,
                     elapsed=elapsed,
                     steps=steps,
                     dx=dxT,
                     dy=dyT,
                     nx=nxT,
                     ny=nyT,
                     sweeps = total_sweeps,
                     tolerance = tolerance)

                
            elif (problem == 'd'):
                print "saving for d!!"
                if dump_to_file: filename = '1d_{0}_step{1}_data_time-{2:.2f}.mpz.npz'.format(N, str(steps).rjust(6, '0'), elapsed)
                else: filename = '1d_{0}_step{1}_data_time-{2:.2f}.npz'.format(N, str(steps).rjust(6, '0'), elapsed)
                #save_data(elapsed, c_var, )

                np.savez(os.path.join(filepath,filename),
                     c_var_array=np.array(c_var),
                     dt=dt,
                     elapsed=elapsed,
                     steps=steps,
                     dx=c_var.mesh.dx,
                     dy=c_var.mesh.dy,
                     nx=c_var.mesh.nx,
                     ny=c_var.mesh.ny,
                     sweeps = total_sweeps,
                     tolerance = tolerance)

        dt, dt_old, dump_times, dump_to_file, filename = calc_dt(elapsed, dt, dt_old, dump_to_file, dump_times, filename)
        steps += 1
        elapsed += dt
        c_var.updateOld()
    else:
        dt *= 0.8
        c_var[:] = c_var.old

# simulation ends
print 'steps reached:', steps


Let's create the json parameter file

In [ ]:
def ParameterFile(N, steps, duration, sweeps, problem, tolerance, solver):
    
    params = {'N' : N,  'steps' : stp, 'sweeps' : sweeps, 'duration':duration, 'problem':problem, 'tolerance':tolerance, 'solver':solver}

    with open('params.json', 'w') as fp:
        json.dump(params, fp)
        

In [ ]:
ParameterFile(50, 200, 200, 2, 'a', 0.1, 'PCG')

# Create a Git Repository 


In this demo, I'm assuming that the working directory is a Git repository set up with

$ git init

gitaddfipytiming.py git ci -m "Add timing script."

Sumatra requires that the script is sitting in the a working copy of a repository.

# Configure Sumatra


Once the repository is setup, the Sumatra repository can be configured. Here we are using the serial launch mode.

In [ ]:
%%bash

\rm -rf .smt #removes any existing sumatra project
smt init smt-fipy-demo #chnage this name to whatever you want
smt configure --executable=python --main=fipy_hackathon_1.py
smt configure --launch_mode=serial
smt configure -g uuid
smt configure -c store-diff
smt configure --addlabel=parameters

# If you want to run Sumatra and store records on the CoRR site

First you have to install the proper version of Sumatra to handle the httpStore methods for sotring on CoRR (Contact Faical Yannick Congo at NIST)

Create an account on CoRR and find the setting gear at the bottom right, and click on the account button, then copy the 'apt key' that is associated with your profile
replace the initializing command with the one here, the others can stay the same

NOTE: sumatra's tag function does not work when using CoRR to store the records

then run this command to initialize:

!smt init -s http://10.200.95.215/api/v0.1/private/"[USER-IDKEY-PROVIDED-BY-CORR]" smt-fipy-demo

Run the simulation using Sumatra

In [ ]:
#This command allows you to change the parameters from the params.json file to anything
#Sumatra creates a new json file when this happens, which contains the new specified parameters

!smt run params.json N=50 steps=200 sweeps=2 duration=200 problem="a" tolerance=0.1 solver="PCG"


once the simulation is done and Sumatra saves the record, we can export the simulation information

In [ ]:
import pandas

#export the sumatra simulation record
!smt export  

#save the record into a blank variable called 'data'
#in your working Sumatra directory, there will be a .smt/ directory which stores all the records
with open('.smt/records_export.json') as ff:
    data = json.load(ff)
    
#save a back up file with a more readable format    
with open('record1.json', 'w') as record1:
    for entry in range(len(data)):
        record1.write(json.dumps(data[entry], sort_keys=True, indent=4, separators=(',', ': ')))
        
#create a dataframe with which we can now work with
df = pandas.DataFrame(data)  #df is now the sumatra dataframe

If you have run simulation in different directories and wish to also include their data with the simulations in your current directory, you can run "smt export" in that directory from the command line. Then open the "directorypath/.smt/records_export.json" file and save it as df2. Later on you can combine the two dataframe into one.

In [ ]:
print df['label'] #to show the Sumatra labels of the simulations in the record

df #to see the whole dataframe

# Extractor() : 

This method finds the simulations which have ".mpz.npz" files in their Data/ folder and gets all the information out of those files into a dataframe. 
First you create a dictionary and fill it with all the information from a single simulation, then add the dictionary to the dataframe as a single row.

Repeat in a loop for every simulation


In [ ]:
# p = str(df['datastore'][0]['parameters']['root'])
# p = p[:-13]
# print p
# datapath = os.path.join(p, '3ea82cfb13ea')
# mfile = glob.glob('{0}/*.mpz.npz'.format(datapath))

# # for mpzfile in mfile:
# fn = np.load(mfile[0])
# for item in fn:
#     print item, ' = ', fn[item]
        
# print mfile
dfPCG['label']

In [ ]:
import glob
import os
import fipy as fp

###FREE ENERGY MATH
results = {}
c_alpha = 0.3
c_beta = 0.7
kappa = 2.0
# M = 5.0
# c_0 = 0.5
# epsilon = 0.01
rho_s = 5.0

#record the volume integral of the free energy 
# equivalent to the average value of the free energy for any cell,
# multiplied by the number of cells and the area of each cell
# (since this is a 2D domain)        
# bulk free energy density

def f_0(c):
    return rho_s*((c - c_alpha)**2)*((c_beta-c)**2)
# free energy
def f(c):
    return (f_0(c)+ .5*kappa*(c.grad.mag)**2)

#.mpz.npz are marker npz files which are saved at specific elapsed times. That way you can compare the concentration values for the same problem under different grid sizes etc.
#and every simulation saves the .mpz.npz files at exactly the same times
def extractor(labelz, Lx=200.):  
    rows = 0
    for label in labelz:
        dictt = {} #create a dictionary to fill in with simulation data
        dictt.update({'label':lable})
        output_filepath = str(df['datastore'][0]['parameters']['root'])
        output_filepath = p[:-13] #cuts off the sumatra label from the datapath
        datapath = os.path.join(output_filepath, lbl)
        mfiles = glob.glob('{0}/*.mpz.npz'.format(datapath)) #get the name of the mpz files with data we need

        #create the mesh for simulation associated with this label
        #this mesh will only work for problem 1a since it is the periodic mesh
        #NEED TO ADD other MESHES for problems b and c
        #this code requires an if statement to check for which problem was run in the simulation
        #and then create the corresponding mesh for the problem. The free energy calculation changes for the problems as well
        fn_0 = np.load(mfile[0]) #first file in the simulaition Data directory
        SimMesh = fp.PeriodicGrid2D(nx=fn_0['nx'], ny=fn_0['ny'], dx = Lx / fn_0['nx'], dy = Lx / fn_0['ny'])
        cvar_array = fn_0['c_var_array']
        c_var = fp.CellVariable(value = cvar_array, mesh = SimMesh)
        
        #Now we need to read each output file and add all the information from each file as a seprate row into the dataframe
        if len(mfiles)>=1:
            for mpzfile in mfiles:
                fn = np.load(mpzfile)
                for item in fn:
                    dictt.update({str(item):fn[item]}) #add every variable from the file into the dictionary
                
                #create blank columns for the dataframe to fill in later
                dictt.update({'L1':[]})
                dictt.update({'L2':[]})
                dictt.update({'Linf':[]})
                dictt.update({'real duration':[]})
                dictt.update({'memory usage':[]})
                
                #update the cell variable and append free energy to the dictionary
                cvar_array = fn['c_var_array']
                c_var[:] = cvar_array
                dx = c_var.mesh.dx #Lx / fn['nx']
                dy = c_var.mesh.dx #Lx / fn['ny']
                free_energy_cellVolAvg = np.mean(f(c_var))*SimMesh.numberOfCells*dx*dy
                dictt.update({'free energy at elapsed': free_energy_cellVolAvg }) 
                
                dfC.loc[rows]=dictt        
                rows+=1 #row iterator
    return dfC #dfC is a compact dataframe with all the information we need

#if you wish to combine another dataframe from a separte directory into this dfC dataframe,
#you can use the same extractor method, update the output_filepath, and begin the row iterator at the last row of dfC
#so rows=len(dfC) 

Create the dataframe which we need to fill in with data

In [ ]:
dfC = pandas.DataFrame(columns=['label','real duration', 'memory usage', 'steps','elapsed', 'nx', 'ny', 'solver', 'tolerance', 'free energy at elapsed', 'c_var_array', 'dt', 'dx', 'dy', 'sweeps', 'L1', 'L2', 'Linf'])


Get all the sumatra labels and call the extractor method on them

In [ ]:
labels = df['label']
extractor(labels) 

## Deleting unwanted simulation from the table

If you have unwanted simulations in the dataframe, here is some example code on how to remove them.

In [ ]:
#make a temporary dataframe which only includes simulatios with sweeps between 1 and 8
dfC_sweepTemp = dfC.query('(sweeps == 1) or (sweeps == 2) or (sweeps == 4) or (sweeps == 3) or (sweeps == 5) or (sweeps == 6) or (sweeps == 7) or (sweeps == 8)')
#Then you can also remove a specific simulation by it's sumatra label
dfC_sweepTemp2 = dfC_sweepTemp.query('(label != "509812ef5230")')

#you can also iterate through the rows and change the information in specific cells if needed.
for index, row in dfC_sweepTemp2.iterrows():

    #now fix the elapsed time for the newest simulation, elapsed is now saved properly so I have to adjust back
    if row['label'] == 'a45690313727' or row['label'] == 'ea519a46fe59':
        dfC_sweepTemp2.loc[index, 'elapsed'] = dfC_sweepTemp2.loc[index, 'elapsed'] - dfC_sweepTemp2.loc[index, 'dt'] #this is in case the elapsed times are saved wrong in the mpz.npz files for some simulations

    #you can find a specific point in a simulation by checking all the conditions at which it might be
    if row['label'] == 'ad4019c3836e' and row['steps'] == int(151) and row['sweeps'] == int(8) and row['solver'] == 'PCG':
        dfC_sweepTemp2.loc[index, 'elapsed'] = float(200.0) #set the elapsed time for the specific row to 200.0
        print 'Elapsed time for ', row['nx'], '-', row['sweeps'], ' is now ', dfC_sweepTemp2.loc[index, 'elapsed'] 
        
dfC_sweepTemp2

This command will show you the first 5 rows, adjust as needed if you wish to see specific rows in the dataframe

In [ ]:
dfC_sweepTemp2.loc[0:5]

# c_var Interpolation

c_var is the concentration variable value in every cell of the grid for a simulation. We wish to take the finest mesh with the most sweeps, and say that

mesh has the most accurate values of concentrations. Call this the Best Simulation. Then we want to compare the other grid sizes to that Best Simulation and 

take the norms of the concentration values as a figure of merit for comparison of accuracy.

The method below will take the datafram which is created by the extractor() above and save the interpolation function into the dataframe.

In [ ]:
#We will create keys for every simulation as identifiers
#simulation of grid size 100 with 8 sweeps and PCGSolver will have the key 100-8-PCG
#You can also include elapsed time in the key, to separate the various elapsed times in a simulation
def Figures_of_Merit(dframe, Lx=200.):
    Lx = float(Lx)
    N_Best = 0
    sweeps_Best = 0
    c_var_Best = {}
    results = {}
    simulations = []
    full_keys = []
    key_Best = ''
    for index, row in dframe.iterrows():
        if row['nx'] > N_Best:
            N_Best = row['nx']
            sweeps_Best = row['sweeps']
        elif row['nx'] == N_Best and row['sweeps'] > sweeps_Best:
            sweeps_Best = row['sweeps']
                    
        key_Best = '{0}-{1}'.format(N_Best,sweeps_Best)
        print key_Best

        key = '{0}-{1}-{2}-{3}'.format(row['nx'], row['sweeps'], row['elapsed'], row['solver'])
        print key
        sim_key = '{0}-{1}-{2}'.format(row['nx'], row['sweeps'], row['solver'])
        if sim_key not in simulations: simulations.append(sim_key)
        if key not in full_keys: full_keys.append(key)the
        results[key] = {'c_var': np.array(row['c_var_array'])}
            
    simulations.append(key_Best)
    print 'Best simulation has been found! ------ ', key_Best
    for key, value in results.iteritems():
        mesh_int = fp.Grid2D(nx=N_Best, ny=N_Best, dx=Lx / N_Best, dy=Lx / N_Best)
        N_sweeps_elapsed_solver = key.split('-')
        N_sweeps_elapsed_solver[0] = int(N_sweeps_elapsed_solver[0])
        N_sweeps_elapsed_solver[1] = int(N_sweeps_elapsed_solver[1])
        m = fp.Grid2D(nx=N_sweeps_elapsed_solver[0], ny=N_sweeps_elapsed_solver[0], dx=Lx / N_sweeps_elapsed_solver[0], dy=Lx / N_sweeps_elapsed_solver[0])
        
        v = fp.CellVariable(mesh=m)
        v[:] = value['c_var'][:]
        v_int = fp.CellVariable(mesh=mesh_int)

        v_int[:] = v((mesh_int.x, mesh_int.y), order=1)
        
        elaps_cvar_Best = '{0}-{1}-{2}-{3}'.format(N_Best,N_sweeps_elapsed_solver[1],  N_sweeps_elapsed_solver[2], 'PCG')
        print 'Simulation: ', N_sweeps_elapsed_solver
        print elaps_cvar_Best

        diff_cvar_Best = np.absolute(results[elaps_cvar_Best]['c_var'] - v_int)
#         print 'The diff of {0} with {1}: '.format(key, elaps_cvar_Best), diff_cvar_Best
        value['L1'] = np.linalg.norm(diff_cvar_Best,1)
        value['L2'] = np.linalg.norm(diff_cvar_Best,2)
        value['Linf'] = np.linalg.norm(diff_cvar_Best,np.inf)
        
    for index, row in dframe.iterrows():
            key = '{0}-{1}-{2}-{3}'.format(row['nx'], row['sweeps'], row['elapsed'], row['solver'])
            dframe.loc[index, 'L1'] = results[key]['L1']
            dframe.loc[index, 'L2'] = results[key]['L2']
            dframe.loc[index, 'Linf'] = results[key]['Linf']
            
    print '-'*100

    best_key = '{0}-{1}'.format(N_Best, sweeps_Best)
    print '='*100
    print dframe['L1']
    return best_key, simulations, full_keys, dframe


In [ ]:
best_key, simulations, all_keys, dfC_final = Figures_of_Merit(dfC_sweepTemp2)


# Plot the Norms for various grid sizes to compare how sweeps affect Norms

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import cycle
cycol = cycle('bgrcmk').next

print simulations

best_nx, best_sweeps = best_key.split('-')
temp_frame = dfC_final.query('(nx == {0}) & (sweeps == {1})'.format(best_nx, best_sweeps))
color = cycol()


graph_grids = []
best_key+=str('-LU')
print 'Best simulation key is : ', best_key
for key in simulations:
    if key != best_key:
        print key
        nx, sweeps, solvr = key.split('-')

        color = cycol()
        temp_frame = dfC_final.query('(nx == {0}) & (sweeps == {1})'.format(nx, sweeps))
        if nx not in graph_grids:
            print 'yay'
            graph_grids.append(nx)
            if int(nx)==100:
                grids_100_L1 = temp_frame.plot('elapsed', 'L1', kind='line', ylim=0, c=color, marker='.', 
                                               title='L1 for N = 100', label=key)
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
                grids_100_L1.set_xlabel('Norm (log scale)')                

                grids_100_L2 = temp_frame.plot('elapsed', 'L2', kind='line', ylim=0, c=color, marker='.',
                                               title='L2 for N = 100', label=key)
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
                grids_100_L2.set_xlabel('Norm (log scale)')                

                grids_100_Linf = temp_frame.plot('elapsed', 'Linf', kind='line', ylim=0, c=color, marker='.',
                                                 title='Linf for N = 100', label=key)
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
                grids_100_Linf.xaxis.set_label('Norm (log scale)')                


            if int(nx)==200:
#                 print '200 yay'
                grids_200_L1 = temp_frame.plot('elapsed', 'L1', kind='line', ylim=0, c=color, marker='.', 
                                               title='L1 for N = 200', label=key) #, logy=True
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
            
                grids_200_L2 = temp_frame.plot('elapsed', 'L2', kind='line', ylim=0, c=color, marker='.', 
                                               title='L2 for N = 200', label=key) #, logy=True
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
                
                grids_200_Linf = temp_frame.plot('elapsed', 'Linf', kind='line', ylim=0, c=color, marker='.', 
                                               title='Linf for N = 200', label=key) #, logy=True
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
                
            if int(nx)==400:
#                 print '400 yay'
                grids_400_L1 = temp_frame.plot('elapsed', 'L1', kind='line', ylim=0, c=color, marker='.', 
                                               title='L1 for N = 400', label=key)
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
            
                grids_400_L2 = temp_frame.plot('elapsed', 'L2', kind='line', ylim=0, c=color, marker='.', 
                                               title='L2 for N = 400', label=key)
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
                
                grids_400_Linf = temp_frame.plot('elapsed', 'Linf', kind='line', ylim=0, c=color, marker='.', 
                                               title='Linf for N = 400', label=key)
                plt.ylabel('Norm (log scale)')
                plt.xlabel('Elapsed Time')
        else:
            if int(nx)==100:
#                 print 'eleeeeeeeeee yay'
                temp_frame.plot('elapsed', 'L1', ax = grids_100_L1, kind='line', ylim=0, c=color, marker='.', 
                                               title='L1 for N = 100', label=key)
                temp_frame.plot('elapsed', 'L2', ax = grids_100_L2, kind='line', ylim=0, c=color, marker='.', 
                                               title='L2 for N = 100', label=key)
                temp_frame.plot('elapsed', 'Linf', ax = grids_100_Linf, kind='line', ylim=0, c=color, marker='.', 
                                                title='Linf for N = 100', label=key)
            
                axes100 = plt.gca()
                # recompute the ax.dataLim
                axes100.relim()
                # update ax.viewLim using the new dataLim
                axes100.autoscale_view()
                plt.draw()

            if int(nx)==200:
                temp_frame.plot('elapsed', 'L1', ax = grids_200_L1, kind='line', ylim=0, c=color, marker='.', 
                                               title='L1 for N = 200', label=key)
                axes200L1 = grids_200_L1.get_axes()
                # recompute the ax.dataLim
                axes200L1.relim()
                # update ax.viewLim using the new dataLim
                axes200L1.autoscale_view()
                plt.draw()
                
                
                temp_frame.plot('elapsed', 'L2', ax = grids_200_L2, kind='line', ylim=0, c=color, marker='.', 
                                               title='L2 for N = 200', label=key)
                temp_frame.plot('elapsed', 'Linf', ax = grids_200_Linf, kind='line', ylim=0, c=color, marker='.', 
                                               title='Linf for N = 200', label=key)
            if int(nx)==400:
                temp_frame.plot('elapsed', 'L1', ax = grids_400_L1, kind='line', ylim=0, c=color, marker='.', 
                                               title='L1 for N = 400', label=key)
                temp_frame.plot('elapsed', 'L2', ax = grids_400_L2, kind='line', ylim=0, c=color, marker='.', 
                                               title='L2 for N = 400', label=key)
                temp_frame.plot('elapsed', 'Linf', ax = grids_400_Linf, kind='line', ylim=0, c=color, marker='.', 
                                               title='Linf for N = 400', label=key)

axplt = plt.gca()
axplt
grids_100_L1.xaxis.set_label('Norm (log scale)')                
plt.ylabel('Norm (log scale)')
plt.xlabel('Elapsed Time')
# plt.legend()
plt.show()
            

# Add memory and real runtime data and save out the dataframe

In order to get the memory usage data, you will have to run the mprofile tool in Linux. You can run the fipy_hackathon_1.py file for a short time, maybe 200 steps and it will record the mmeory usage data from it. Then you can get the maximum memory peak as the max CPU usage.

Here is the code for how to extract the maximum peaks from the mprofile output files:


In [ ]:
mprofFiles = glob.glob('*.dat') #get the list of .dat memory profile files
memory_dictionary = {'50': None, '100': None, '200': None, '400': None}
if len(mprofFiles)>=1:
    for memFile in mprofFiles:
        ff = open(memFile, 'r')
        data = ff.read()
        data_list = data.split('\n')
        data_sublist = [line.split(' ') for line in data_list[1:]] #ignores the first entry because it's just text
        memdata = [float(item[1]) for item in data_sublist[:-1]] #ignores the last entry, its just a space
        memoryPeak = max(memdata) #we have the memory peak for the simulation 
        print memoryPeak
        #now to save it in the proper dictionary entry
        gh = memFile.split('_')
        size = gh[1].split('N')[1] #split and get the grid size from file name
        memory_dictionary[size] = memoryPeak
        
print memory_dictionary

with open('/data/aem1/new1a/memory-profiles/memory_peaks_LU.json', 'w') as fp:
    json.dump(memory_dictionary, fp)

add the memory usage data and real world duration times

In [ ]:
#what are the simulations we are concerned with? better to use the final dataframe in case some simulation were not included from original sumatra dataframe
temp_labels = []
for thing in np.unique(dfC_final['label']):
    print str(thing)
    temp_labels.append(str(thing))


with open('memory_peaks_LU.json') as ff_LU:
    memory_data_LU = json.load(ff_LU)
with open('memory_peaks_PCG.json') as ff_PCG:
    memory_data_PCG = json.load(ff_PCG)
    
print memory_data_LU #this is the memory data we will use for all simulations
print memory_data_PCG
print '-'*100


for index, row in dfC_final.iterrows():
    if row['solver']=="LU":
        lbl = str(row['label']) #get the sumatra label
        dfC_final.loc[index, 'memory usage'] = memory_data_LU[str(row['nx'])] #input the memory usage data that matches the grid size in the memory data dictionary
        dfC_final.loc[index, 'real duration'] = float(df.query('(label=="{0}")'.format(lbl))['duration']) #input the real world time data from the original sumatra dataframe
    elif row['solver']=="PCG":
        lbl = str(row['label'])
        dfC_final.loc[index, 'memory usage'] = memory_data_PCG[str(row['nx'])]
        dfC_final.loc[index, 'real duration'] = float(df.query('(label=="{0}")'.format(lbl))['duration'])

In [ ]:
dfC1_dropcol = dfC_final.drop('c_var_array', 1) #drop the array to save space
dfC1_dropcol.to_csv('dfC_final_data.csv') #save the file to email to daniel

# Method for Time & Free Energy lists

The code below will get a list of free energy values and a list of elapsed time values for a specified sumatra label

In [ ]:
def f_0(c):
    return rho_s*((c - c_alpha)**2)*((c_beta-c)**2)
# free energy
def f(c):
    return (f_0(c)+ .5*kappa*(c.grad.mag)**2)

def EnergyVStimeLists(sumatraLabel):
    results = {}
    c_alpha = 0.3
    c_beta = 0.7
    kappa = 2.0
    M = 5.0
    c_0 = 0.5
    epsilon = 0.01
    rho_s = 5.0

    energyList = []
    timeList = []
    Lx=200.0
    #This code will access every step file saved in the Data/[label] directory 
    
    index = df.label[df.label == str(sumatraLabel)].index.tolist() #this should be a list of a single row index for the specified sumatra label
    if len(index) == 1:
        output_filepath = df['datastore'][index[0]]['parameters']['root'] 

    stepfiles = glob.glob('{0}/*.npz'.format(filepath)) #get the list of all step files for simulation

    fn_0 = np.load(stepfiles[0])
    SimMesh = fp.PeriodicGrid2D(nx=fn['nx'], ny=fn['ny'], dx = Lx / fn['nx'], dy = Lx / fn['ny'])
    cvar_array = fn_0['c_var_array']
    c_var = fp.CellVariable(value = cvar_array, mesh = SimMesh)

    for stpfile in stepfiles:
        fn = np.load(stpfile)

        cvar_array = fn['c_var_array']
        c_var[:] = cvar_array
        cells = fn['nx']*fn['ny']
        dx = Lx / fn['nx']
        dy = Lx / fn['ny']
        free_energy_array = f(c_var)
        free_energy_cellVolAvg = np.mean(free_energy_array)*cells*dx*dy
        energyList.append(free_energy_cellVolAvg)
        timeList.append(float(fn['elapsed']))
        
    return timeList, energyList


In [ ]:
import matplotlib.pyplot as plt
plt.plot(timeList, energyList)
plt.show()

# Free Energy Interpolation

For this section, we have to take the elapsed time and cvar from each step file, calculate the energy at that step, then interpolate the array of energies along with time. Save the intepolated function into the dataframe

In [ ]:
import os
import numpy as np
from scipy import interpolate as scpinter

results = {}
c_alpha = 0.3
c_beta = 0.7
kappa = 2.0
# M = 5.0
# c_0 = 0.5
# epsilon = 0.01
rho_s = 5.0

def f_0(c):
    return rho_s*((c - c_alpha)**2)*((c_beta-c)**2)
def f(c):
    return (f_0(c)+ .5*kappa*(c.grad.mag)**2)


def freeEnergyInterp(labels, dframe, Lx = 200.):
    test_dict = {}
    energyFunctions = []
    #This code will access every step file saved in the Data/[label] directory 
    for label in dframe['label']:
        e_elaps = []
        e_enrg = []
        dpath = str(df['datastore'][0]['parameters']['root'])
        dpath = dpath[:-13]
        filepath = os.path.join(dpath, label)

        stepfiles = glob.glob('{0}/*.npz'.format(filepath)) #get the list of all step files for simulation
        if len(stepfiles)==0: 
            energyFunctions.append(None)
        else:
            fn_0 = np.load(stepfiles[0])
            SimMesh = fp.PeriodicGrid2D(nx=fn_0['nx'], ny=fn_0['ny'], dx = Lx / fn_0['nx'], dy = Lx / fn_0['ny'])
            cvar_array = fn_0['c_var_array']
            c_var = fp.CellVariable(value = cvar_array, mesh = SimMesh)
            for stpfile in stepfiles:
                fn = np.load(stpfile)

                cvar_array = fn['c_var_array']
                c_var[:] = cvar_array
                cells = fn['nx']*fn['ny']
                dx = Lx / fn['nx']
                dy = Lx / fn['ny']
                free_energy_array = f(c_var)
                free_energy_cellVolAvg = np.mean(free_energy_array)*cells*dx*dy


                e_elaps.append(fn['elapsed'])
                e_enrg.append(free_energy_cellVolAvg)
        print 'e_elaps: ', len(e_elaps), '   e_enrg: ', len(e_enrg)
        eFunction = scpinter.interp1d(e_elaps, e_enrg, copy=False)
        energyFunctions.append(eFunction) #save a list of all the energy functions
        print ' '
        print '='*100
        print 'Number of energy functions: ', len(energyFunctions) 
        
        
        #svae values into a dictionary to figure out why norms are 0 for grids 400
        if label not in test_dict.keys():
            test_dict[label] = {'elapsed':e_elaps, 'energy':e_enrg}
            
            
            
    dframe['Free_Energy_Interpolated_Function'] = energyFunctions #add the list as a new column to the dataframe
    return test_dic


In [ ]:
#Now run the method above
labels = dfC_final['label']
test_dict = freeEnergyInterp(labels, dfC_final)

# Free Energy norms from the Interpolated functions

Now we need to get the free energy function from the best simulation, and do norm calculations with it compared to the other simulations. 

In [ ]:
#This method takes any set of "ideal times" and compares the free energy values at those times for every simulation
#Create the lists of norms, then insert the lists into the columns of the dataframe

ideal_times = []
for i in xrange(1, 1000): ideal_times.append(i)
#the range of ideal times cannot exceed the maximum time of the shortest simulation. So if you have simulations that ran until 1000, 1500, 3000 elapsed time
#you cannot exceed 1000 for the maximum ideal time.
    
def FreeEnergyCompare(dframe, ideal_times):
    L1Norms = [] 
    L2Norms = []
    LinfNorms = []
    N_Best = 0
    sweeps_Best = 0
    
    for row in dframe.iterrows():
        if row[1]['nx'] >= N_Best and row[1]['sweeps'] >= sweeps_Best:
            BestEnergyFunct = row[1]['Free_Energy_Interpolated_Function']
            N_Best = row[1]['nx']
            sweeps_Best = row[1]['sweeps']
    for row in dframe.iterrows():    
        efunction = row[1]['Free_Energy_Interpolated_Function']
        diff = np.absolute(efunction(ideal_times) - BestEnergyFunct(ideal_times))
        L1Norms.append(np.linalg.norm(diff,1)) 
        L2Norms.append(np.linalg.norm(diff,2)) 
        LinfNorms.append(np.linalg.norm(diff, np.inf)) 
    
    
    dframe['L1 Free Energy Norms'] = L1Norms
    dframe['L2 Free Energy Norms'] = L2Norms
    dframe['L-infinite Free Energy Norms'] = LinfNorms

FreeEnergyCompare(dfC_final, ideal_times)

# Memory vs CPU time graphing

Now we will plot Memory Usage vs CPU for various grid sizes 

the color of the dot corresponds to how small the Norm for that grid size is (green = smallest)

# Create a dataframe for the memory data to graph it

In [ ]:
temp_labels = []
for thing in np.unique(dfC_final['label']):
    print str(thing)
    temp_labels.append(str(thing))
    
print temp_labels

In [ ]:
#what are the simulations we are concerned with?
sim_labels = temp_labels
dfMem = pandas.DataFrame(columns=['label','steps','sumatra duration', 'elapsed', 'nx', 'sweeps', 'memory usage', 'L1 Free Energy', 'L2 Free Energy', 'Linf Free Energy'])
rows = 0
data_dict = {} #the dictionary to which we will add data

with open('memory_peaks.json') as ff:
    memory_data = json.load(ff)
print memory_data #this is the memory data we will use for all simulations
    
for label in sim_labels:
    temp_df = df.query('(label == "{0}")'.format(label))
    #build the dictionary to add to the memory dataframe
    data_dict['sumatra duration'] = float(temp_df['duration'])
    data_dict['label'] = label   
    data_dict['steps'] = 0
    data_dict['elapsed'] = 0
    data_dict['nx'] = int(dfC_final.query('(label=="{0}")'.format(label)).iloc[0]['nx'])
    data_dict['sweeps'] = int(dfC_final.query('(label=="{0}")'.format(label)).iloc[0]['sweeps'])
    data_dict['memory usage'] = memory_data[str(data_dict['nx'])]
    data_dict['L1 Free Energy'] = float(dfC_final.query('(label=="{0}")'.format(label)).iloc[0]['L1 Free Energy Norms'])
    data_dict['L2 Free Energy'] = float(dfC_final.query('(label=="{0}")'.format(label)).iloc[0]['L2 Free Energy Norms'])
    data_dict['Linf Free Energy'] = float(dfC_final.query('(label=="{0}")'.format(label)).iloc[0]['L-infinite Free Energy Norms'])
    
    dfMem.loc[rows]=data_dict
    rows +=1
    
dfMem

In [ ]:
memory_plot = dfMem.plot('sumatra duration', 'memory usage', kind='scatter',ylim=0, title='CPU vs Memory', c=dfMem['Linf Free Energy'], cmap='RdYlGn_r', s=40)

fig = plt.figure()
axPlot = fig.add_subplot(111, axes = memory_plot.axes)

axPlot.plot(dfMem['sumatra duration'], dfMem['memory usage'],'o')
plt.ylabel('Peak Memory Usage (MiB)')
memory_plot.set_xlabel('Simulation Duration (s)')
memory_plot.axes.set_xlabel('Simulation Duration (s)')
plt.subplots_adjust(bottom=0.15)

plt.show()
print memory_plot.axes.xaxis.label
